In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import cv2
import os

In [3]:
lables = pd.read_csv('data/train_ship_segmentations_v2.csv')

In [4]:
lables.describe()

,ImageId,EncodedPixels
count,231723,81723
unique,192556,81722
top,e6fd0c12e.jpg,43801 1 44567 4 45334 5 46100 8 46867 9 47636 ...
freq,15,2


In [5]:
lables.head()

,ImageId,EncodedPixels
0,00003e153.jpg,NaN
1,0001124c7.jpg,NaN
2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...


In [6]:
print(f"Percentage of images with ship: {(lables.EncodedPixels.count()/len(lables))*100:.2f}%")

Percentage of images with ship: 35.27%


In [7]:
img_sizes = []
img_names = []
for i in os.listdir("data/train_v2"):
    img_names.append(i)
    img_sizes.append(cv2.imread("data/train_v2/"+i).shape)

In [5]:
os.listdir("data/train_v2")

['00003e153.jpg',
 '0001124c7.jpg',
 '000155de5.jpg',
 '000194a2d.jpg',
 '0001b1832.jpg',
 '00021ddc3.jpg',
 '0002756f7.jpg',
 '0002d0f32.jpg',
 '000303d4d.jpg',
 '00031f145.jpg',
 '00052ed46.jpg',
 '000532683.jpg',
 '00053c6ba.jpg',
 '00057a50d.jpg',
 '000592296.jpg',
 '0005d01c8.jpg',
 '0005d6d95.jpg',
 '0006c52e8.jpg',
 '0007b8229.jpg',
 '000805313.jpg',
 '000811bb6.jpg',
 '0008d6216.jpg',
 '000913daf.jpg',
 '000952d51.jpg',
 '000969125.jpg',
 '0009a46e1.jpg',
 '000a4d4fd.jpg',
 '000aed819.jpg',
 '000baef0c.jpg',
 '000bd9ac4.jpg',
 '000c150b0.jpg',
 '000c34352.jpg',
 '000d26c17.jpg',
 '000d42241.jpg',
 '000db3e62.jpg',
 '000e37fc6.jpg',
 '000e6378b.jpg',
 '000e64855.jpg',
 '000f1f959.jpg',
 '000f7e728.jpg',
 '000f7f2fd.jpg',
 '000fd9827.jpg',
 '00104f16f.jpg',
 '0010e88ce.jpg',
 '0010eefb1.jpg',
 '00113a75c.jpg',
 '00119e4ba.jpg',
 '0011a9ccb.jpg',
 '0011c31b7.jpg',
 '00122061c.jpg',
 '001234638.jpg',
 '0012669a8.jpg',
 '0012a8fa1.jpg',
 '0012b8981.jpg',
 '0012be3bb.jpg',
 '0013b222